In [ ]:
pip install --upgrade google-api-python-client

In [ ]:
pip install oauth2client

In [1]:
from googleapiclient.http import MediaIoBaseDownload,MediaFileUpload
from googleapiclient.discovery import build
import pprint
import os
import httplib2 
import requests
from oauth2client.service_account import ServiceAccountCredentials
import urllib.request
from requests.exceptions import HTTPError

pp = pprint.PrettyPrinter(indent=4)

In [2]:
SCOPES = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']
SERVICE_ACCOUNT_FILE = 'task2-download.json'
spreadsheet_id = '1A6XaXqjBvh78RvLkCjSu8C6MNlxMcGlU67L6R_8tpDI'

In [3]:
credentials = ServiceAccountCredentials.from_json_keyfile_name(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)
httpAuth = credentials.authorize(httplib2.Http())
service = build('sheets', 'v4', http = httpAuth)

In [4]:
spreadsheet = service.spreadsheets().get(spreadsheetId = spreadsheet_id).execute()
sheetList = spreadsheet.get('sheets')
sheet_name, sheet_id = [], []
for sheet in sheetList:
    sheet_name.append(sheet['properties']['title'])
    sheet_id.append(sheet['properties']['sheetId'])

In [5]:
def get_values(ranges, spreadsheetId):   
    results = service.spreadsheets().values().batchGet(spreadsheetId = spreadsheetId, 
                                     ranges = ranges, 
                                     valueRenderOption = 'FORMATTED_VALUE',  
                                     dateTimeRenderOption = 'FORMATTED_STRING').execute() 
    return results['valueRanges'][0]['values']

In [6]:
def update_values(ranges, values, spreadsheetid, services):
    values = services.spreadsheets().values().batchUpdate(spreadsheetId = spreadsheetid,
                                                   body = {'valueInputOption':'USER_ENTERED',
                                                           'data' : [{'range': ranges,
                                                                      'majorDimension':'COLUMNS',
                                                                      'values': values}
                                                                    ]
                                                          }
                                                  ).execute()
    return values

In [7]:
path_folder_for_zip = 'C://Users/strelokask/Desktop/Amina/_project2/files'
if os.path.exists(path_folder_for_zip):
    print("Указанный файл существует") 
else:
    os.mkdir(path_folder_for_zip)

Указанный файл существует


In [8]:
folder_start_column = 'A'
folder_start_row = '2'
url_start_column = 'B'
url_start_row = '2'
max_number_of_urls = '7'
max_folder_count = '699'

url_end_ = chr(ord(url_start_column) + int(max_number_of_urls))
folder_start = folder_start_column + folder_start_row
url_start = url_start_column + url_start_row
range_ = chr(ord(folder_start_column) + int(max_number_of_urls)) + max_folder_count

In [14]:
folders, count_all_img = [], []
for folder_name in sheet_name:  
    if os.path.exists(path_folder_for_zip + '/' + folder_name):
        print("Указанный файл существует") 
    else:
        os.mkdir(path_folder_for_zip + '/' + folder_name)
    count_img = []    
    path_of_directory_root = path_folder_for_zip + '/' + folder_name + '/'
    ranges = [folder_name + '!' + folder_start + ':' + range_]  
    sheet_values = get_values(ranges, spreadsheet_id)
    for elem in sheet_values:  
        excess = 1
        path_of_directory = path_of_directory_root + elem[0]
        folders.append(elem[0])
        if os.path.exists(path_of_directory):
            print("Указанный файл существует") 
        else:
            os.mkdir(path_of_directory)
        count = len(elem) - 1
        for i in range(1, count):
            url = 'http://' + elem[i]
            try:
                response = requests.get(url)
                response.raise_for_status()
            except HTTPError as http_err:
                count -= 1
                print(f'HTTP error occurred: {http_err}')
            except Exception as err:
                count -= 1
                print(f'Other error occurred: {err}')
            else:
                path_for_image = path_of_directory + '/' + str(i) + '.jpg'
                with open(path_for_image, "wb") as f:
                    f.write(response.content)
        count_img.append(count)
    count_all_img.append(count_img)

In [26]:
folders = []
path_for_image =''
count_all_img = []
url_list = []
for folder_name in sheet_name:  
    count_img = []    
    excess = 1
    path_of_directory_root = path_folder_for_zip + '/' + folder_name + '/'
    ranges = [folder_name + '!' + folder_start + ':' + range_]  
    sheet_values = get_values(ranges, spreadsheet_id)
    for elem in sheet_values:
        count = len(elem) - 1
        for i in range(1, count):
            if len(elem[i]) < 1:
                count -= 1
        count_img.append(count)
    count_all_img.append(count_img)

In [25]:
for i in range(len(sheet_name)):
    count = str(len(count_all_img[i]) + 1)
    range_sheet = sheet_name[i] + '!' + url_end_ + url_start_row + ':' + url_end_ + '3'
    update_values(range_sheet, [count_all_img[i]], spreadsheet_id, service)